In [1]:
import os
import timm
import torch
import open_clip
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from transformers import AutoModel
import torchvision.transforms as T
from torchvision import transforms
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, balanced_accuracy_score, matthews_corrcoef

In [2]:

file_path = "/fs/ess/PAS2136/Hawaii-2025/beetles_intake/BeetlePalooza Data/Benchmarking/Beetlepalooza_beetles_image_measurements.csv"

df = pd.read_csv(file_path, sep="\t")


In [3]:
df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width
0,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,1.117336,0.637281
1,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Chlaenius,Chlaenius aestivus,1.068345,0.627180


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModel.from_pretrained("facebook/dinov2-base").to(device)


preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
def extract_features(image_path):
    
    image = Image.open(image_path).convert("RGB")
    
    image_tensor = preprocess(image).unsqueeze(0).to(device)

    with torch.no_grad():
        features = model(image_tensor).last_hidden_state.mean(dim=1)

    
    return features.cpu().numpy()
    

In [6]:
df['Embedding'] = [extract_features(img) for img in tqdm(df['ImageFilePath'], desc='Extracting Embeddings')]

X = np.vstack(df['Embedding'].values)
X = np.hstack((X, df[['Length', 'Width']].values))

label_encoder = LabelEncoder()

y = label_encoder.fit_transform(df['ScientificName'])


Extracting Embeddings: 100%|██████████| 11399/11399 [16:36<00:00, 11.44it/s]


In [7]:
df_indices = df.index 

X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df_indices, test_size=0.2, random_state=42)

test_df = df.loc[test_idx].copy()

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)


In [8]:
seed = 99

models = {
    "NaiveBayes": GaussianNB(),
    "LogisticRegression": LogisticRegression(max_iter=100),
    "NearestNeighbor": KNeighborsClassifier(n_neighbors=5),   
    "MLP_Baseline": MLPClassifier(hidden_layer_sizes=(100,), max_iter=200, random_state=seed)
}

predictions = {}

metrics = {}

for name, model in models.items():
    
    model.fit(X_train_scaled, y_train)
    preds = model.predict(X_test_scaled)
    predictions[name] = preds
    
    acc = accuracy_score(y_test, preds)
    prec = precision_score(y_test, preds, average="weighted")
    rec = recall_score(y_test, preds, average="weighted")
    f1 = f1_score(y_test, preds, average="weighted")
    bal_acc = balanced_accuracy_score(y_test, preds)
    mcc = matthews_corrcoef(y_test, preds)
    
    metrics[name] = {"Model": name, "Accuracy": acc, "Precision": prec, "Recall": rec, "F1-Score": f1, "Balanced Acc": bal_acc, "MCC": mcc}
    print(f"{name:<25} | Acc: {acc:.2%} | Prec: {prec:.2%} | Rec: {rec:.2%} | F1: {f1:.2%} | Bal Acc: {bal_acc:.2%} | MCC: {mcc:.4f}")


metrics_df = pd.DataFrame(metrics).T

/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NaiveBayes                | Acc: 65.22% | Prec: 73.50% | Rec: 65.22% | F1: 67.60% | Bal Acc: 56.66% | MCC: 0.6346


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


LogisticRegression        | Acc: 94.78% | Prec: 94.45% | Rec: 94.78% | F1: 94.30% | Bal Acc: 77.09% | MCC: 0.9441


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


NearestNeighbor           | Acc: 79.25% | Prec: 77.41% | Rec: 79.25% | F1: 77.26% | Bal Acc: 48.42% | MCC: 0.7769
MLP_Baseline              | Acc: 94.34% | Prec: 93.65% | Rec: 94.34% | F1: 93.52% | Bal Acc: 73.88% | MCC: 0.9394


/users/PAS2136/rayees/myenv/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [9]:
test_df = test_df.assign(**{f"Pred_{name}": label_encoder.inverse_transform(pred) for name, pred in predictions.items()})
test_df.head(2)

,ImageFilePath,Genus,ScientificName,Length,Width,Embedding,Pred_NaiveBayes,Pred_LogisticRegression,Pred_NearestNeighbor,Pred_MLP_Baseline
1766,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Synuchus,Synuchus impunctatus,0.530811,0.242519,"[[-0.15419693, -0.5175861, 0.95667034, 0.33132...",Cymindis cribricollis,Synuchus impunctatus,Synuchus impunctatus,Synuchus impunctatus
2629,/fs/ess/PAS2136/Rayeed/BeetlePalooza/individua...,Agonum,Agonum punctiforme,0.457965,0.243925,"[[-0.40635183, -1.0228174, 0.54247725, 0.23327...",Agonum punctiforme,Calathus advena,Synuchus impunctatus,Synuchus impunctatus


In [10]:
test_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Full/DINOv2-traits.csv", index=False)
metrics_df.to_csv("/users/PAS2136/rayees/CV4A - MultiModality/Full/DINOv2-traits-metrics.csv", index=False)